# Attention メカニズムのコーディング

## 3.1. 長いシーケンスをモデル化するときの問題点

再起型ニューラルネットワーク (recurrent neural network: RNN): 前ステップの出力を入力に受け取る．エンコーディング中の過去の隠れ層にアクセスできない問題があった．

## 3.2. Attention メカニズムでデータの依存関係をとらえる

Bahdanau Attention メカニズム: デコーディングステップで入力シーケンスの異なる部分に選択的にアクセスできる．  
Self-Attention: 出力シーケンスの各値の計算時に、入力シーケンス内の位置が同じ成分に対して他のすべての位置の関連性を考慮できるようにするメカニズム．

## 3.3. Self-Attention を使って入力の異なる部分に注意を払う

### 3.3.1. 訓練可能な重みをもたない単純な Self-Attention メカニズム

In [2]:
import torch
# トークンの埋め込みの例
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your(x^1)
     [0.55, 0.87, 0.66], # Journey (x^2)
     [0.57, 0.85, 0.64], # starts (x^3)
     [0.22, 0.58, 0.33], # with (x^4)
     [0.77, 0.25, 0.10], # one (x^5)
     [0.05, 0.80, 0.55]] # step (x^6)
)

埋め込み後の一つのトークン `query` と、それを含むバッチ `inputs` に着目して、バッチ内の各成分ごとに内積を計算してみる．

In [3]:
query = inputs[1]
attn_score_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_score_2[i] = torch.dot(x_i, query) # x_i(=inputs[i]) と query(=inputs[1]) のベクトルの内積

print(attn_score_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


attention weights (`attn_weights_2_t,p`) は attension socore (`attn_score_2`) を総和で割ることで、その総和を 1 に調整したもの

In [4]:
attn_weights_2_tmp = attn_score_2 / attn_score_2.sum()
print(f"Attention weights: {attn_weights_2_tmp.__repr__()}")
print(f"Sum: {attn_weights_2_tmp.sum().__repr__()}")

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


実際には softmax 関数で正規化する。 -> 最適化の勾配を安定させる。

$$ \text{Softmax}(z) = \left(\frac{e^{z_i}}{\sum_{j=1}^{n} e^{z_j}}\right)_{i=1 \dots n} $$
ソフトマックス関数で計算した重みは重要度として解釈できる．

In [5]:
def softmax_naive(x): # 素朴実装はオーバーフロー、アンダーフローに弱い
    return torch.exp(x) / torch.exp(x).sum(dim=0)

In [6]:
attn_weights_2_naive = softmax_naive(attn_score_2)
print(f"Attention weights: {attn_weights_2_naive.__repr__()}")
print(f"Sum: {attn_weights_2_naive.sum().__repr__()}")

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [7]:
attn_weights_2 = torch.softmax(attn_score_2, dim=0)
print(f"Attention weight: {attn_weights_2.__repr__()}")
print(f"Sum: {attn_weights_2.sum().__repr__()}")

Attention weight: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [8]:
query = inputs[1] # 2つ目の入力トークンがクエリのとき、出力は
context_vec_2 = torch.zeros(query.shape)

for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2. すべての入力トークンについて Attention の重みを計算する

In [9]:
attn_scores = torch.empty(6,6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


この計算結果は、行列とその行列の転置の積に他ならない．

In [10]:
attn_socres = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [11]:
attn_weights = torch.softmax(attn_scores, dim=1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [12]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)
print("All row sums: ", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums:  tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [13]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)
print("Previous 2nd context vector:", context_vec_2)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])
Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4. 訓練可能な重みを持つ Self-Attention を実装する

入力要素 $x$ に対して、クエリベクトル、キーベクトル、値ベクトルを計算する。正確には、それぞれ行列 $W_q$, $W_k$、$W_v$ を使って、入力行列 $x$ との行列式 $xW_q$、
$xW_k$、$xW_v$ でそれぞれ計算する。$W_q$, $W_k$、$W_v$ を最適化していく必要がある。

- $ \operatorname{Attention}(X) := \operatorname{softmax}(X \cdot W_q^T \cdot (X \cdot W_k^T)^T/\sqrt{d})$
- $ \operatorname{context}(X) := \operatorname{Attention(X)} \cdot (X \cdot W_v^T)$

前節同様に、"Your journey begins one step" というベクトル (トークンベクトル) に対して、文脈ベクトル z_1 ... z_7　を計算する。

まずは `journey` -> z_2 の計算をする

In [14]:
x_2 = inputs[1] # 2 つのめの入力要素 "Journey"
d_in = inputs.shape[1] # 入力埋め込みのサイズ
d_out = 2 # 出力埋込 (本来同じ次元にすべきだが、結果を見やすくするためにあえて d_in と異なる次元にしている)
print(f"x_2 : {x_2}")
print(f"d_in : {d_in}")
print(f"d_out : {d_out}")

x_2 : tensor([0.5500, 0.8700, 0.6600])
d_in : 3
d_out : 2


In [15]:
torch.manual_seed(123) # これから学習するパラメータ
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

In [16]:
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)
print(key_2)
print(value_2)

tensor([0.4306, 1.4551])
tensor([0.4433, 1.1419])
tensor([0.3951, 1.0037])


入力 x の全体に対してクエリベクトル、キーベクトル、値ベクトルを計算する

In [17]:
keys = inputs @ W_key
values = inputs @ W_value
print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


key_2 の Attention 重みは

In [18]:
keys_2 = keys[1]
attn_score_22 = query_2.dot(key_2)
print(attn_score_22)

tensor(1.8524)


全体の Attention 重みは

In [19]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


これを softmax 関数で正規化する

In [20]:
# Attention スコアのスケーリング
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1) # 埋め込み次元の平方根で割っている (勾配を極端にして学習スピードを高める)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


内積で得られるベクトルをスケーリングする操作から、 **Scaled Dot-Prpduct Attention** と呼ばれている。  
最後にコンテキストベクトル z_2 を計算する。

In [21]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2. コンパクトな Self-Attention Python クラスを実装する

In [22]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out)) # torch.rand は指定された形の torch.tensor をランダムな値で生成する
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))
    
    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim =-1
        )
        context_vec = attn_weights @ values
        return context_vec

In [23]:
torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


`torch.nn.Linear` 層を使って上のクラスの実装を簡単にする。
`torch.nn.Linear` は最適化された重み初期化スキームを持っていて明示的に `torch.rand` しなくてよくなっている。

In [24]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias = False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias = qkv_bias)
    
    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        context_vec = attn_weights @ values
        return context_vec

In [25]:
torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


練習問題【解説】  
`sa_v1`, `sa_v2` は それぞれ `torch.Parameter`、`torch.Linear` で行列を管理しているので、型が異なりうまく代入できない。 
`torch.nn.Linear` は `.weight` で重みを `torch.Tensor` 型で保管しており（しかも転置した状態）、これを `torch.Parameter` の型に合わせてやれば `sa_vi` の方の重み行列に代入できる。

In [26]:
print(type(sa_v2.W_key.weight))
print(sa_v2.W_key.weight.T)
print()
print(type(sa_v1.W_key))
print(sa_v1.W_key)

<class 'torch.nn.parameter.Parameter'>
tensor([[ 0.4058,  0.2134],
        [-0.4704, -0.2601],
        [ 0.2368, -0.5105]], grad_fn=<PermuteBackward0>)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([[0.1366, 0.1025],
        [0.1841, 0.7264],
        [0.3153, 0.6871]], requires_grad=True)


In [27]:
sa_v1.W_key = torch.nn.Parameter(sa_v2.W_key.weight.T)
sa_v1.W_query = torch.nn.Parameter(sa_v2.W_query.weight.T)
sa_v1.W_value = torch.nn.Parameter(sa_v2.W_value.weight.T)

print(sa_v1(inputs))
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)
tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


## 3.5 Casual Attention で未来の単語を隠す

- **Self Attention** : 入力シーケンス全体に一度アクセス
- **Casual Attention** : 現在と以前の入力だけにアクセス (Masked Attention とも呼ばれる)

-> Attention 重み行列の対角線よりも上側をマスクすればよい。

### 3.5.1 Casual Attention マスクを適用する

In [28]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)

attn_scores = queries @ keys.T
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

Attention 重みは

In [29]:
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


これの右上半分を 0 にしたい。`torch.tril` を使う。

In [30]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


アダマール演算 (同じサイズの行列に対して各成分の積で定める) は `*` っぽい

In [31]:
masked_simple  = attn_weights * mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


再度各行を正規化（和が1） になるようにする

In [32]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


mask してから softmax して再度 mask しても、softmax してから一度だけ mask しても結果は変わらない. (比が変わらないから)

In [33]:
attn_scores_2 = attn_scores * mask_simple
attn_weights_2 = torch.softmax(attn_scores_2 / keys.shape[-1]**0.5, dim=-1)
masked_simple_2 = attn_weights_2 * mask_simple
row_sums_2 = masked_simple_2.sum(dim=-1, keepdim=True)
masked_simple_norm_2 = masked_simple_2 / row_sums_2
print(masked_simple_norm_2)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


Attention スコアの計算後、上三角の部分を -inf に変換しておくと `-inf` の部分は softmax 計算時に $e^{-\infty}$ として扱われるので、softmax 計算がすでに所望の平均を計算していたことになる。

In [34]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), - torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


実際に各行の和が1であることを確認する

In [35]:
attn_weights = torch.softmax(masked /keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 Attention の重みにドロップアウトマスクを適用する

- **ドロップアウト (dropout)** : ランダムに選択された隠れ層ユニットを無視することで、過剰適合(オーバーフィッティング)を防ぐ

Attention 重みを計算した後にマスクをランダムに適用させる。
※ ベクトル計算後に適用させる流派もあるらしい

ドロップアウト率 50% で実装

In [36]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(p=0.5) # ドロップアウト率 50%
example = torch.ones(6, 6) # 1 埋め行列
print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


成分が2になっているのは、削除された成分の分、スケールアップするため 1/(0.5) = 2 倍されている。  
-> Attention 重みの全体的なバランスを維持できる

In [37]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


### 3.5.3 コンパクトな Casual Attention クラスを実装する

In [38]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch)
print(batch.shape)

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])
torch.Size([2, 6, 3])


In [39]:
class CasualAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
        self.dropout = nn.Dropout(p=dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
            ) # テンソルがモデルパラメータと同じデバイス上にあることを保証するために register_buffer を使用
        # バッファをモデルとともに適切なデバイス(CPU/GPU)に自動的に移動されることがあり、この性質が有効になる

    def forward(self, x):
        b, num_tokens, d_in = x.shape # 次元 1 と 2 を入れ替える。バッチ次元は最初の位置 (0) のまま
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.transpose(1, 2)
        attn_scores.masked_fill_(
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf
            )# <-- `_`終わりのメソッドは、インプレース実行 (オブジェクトの内容を直接変更) され。無駄なメモリコピーを回避する
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights)
        
        context_vec = attn_weights @ values
        return context_vec

In [40]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CasualAttention(
    d_in,
    d_out,
    context_length,
    dropout=0.0,
)

context_vecs = ca(batch)
print(context_vecs)
print("contect_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
contect_vecs.shape: torch.Size([2, 6, 2])


## 3.6 Single-Attention を Multi-head Attention に拡張する

### 3.6.1 複数の Single-head Attention 層を積み重ねる

In [41]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, 
                 qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            CasualAttention(d_in, d_out, context_length, dropout, qkv_bias)
            for _ in range(num_heads)
        )
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1) # 単に結果を結合するだけ

In [42]:
torch.manual_seed(123)
context_length = batch.shape[1]
d_in, d_out = 3, 2
mha = MultiHeadAttention(
    d_in,
    d_out,
    context_length,
    dropout=0.0,
    num_heads=2,
)
context_vecs = mha(batch)

print(context_vecs)
print("contect_vecs_mha.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
contect_vecs_mha.shape: torch.Size([2, 6, 4])


### 3.6.2 重みを分割することで Multi-head Attention を実装する

`CasualAttention` 内の記述を `MultiHeadAttention` に移植、修正する。  
やってることは、行列式の計算階数を for でやっているところを、うまく結合することにより一回の計算で済ませている。

In [43]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), "d_out must be divisible by num_heads"
    
        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # 出力次数をhead で分割    

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        
        self.out_proj = nn.Linear(d_out, d_out) # Linear 層を使ってヘッドの出力を組み合わせる
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length), diagonal=1)
        )
        
    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)
        
        keys = keys.transpose(1, 2) # (b, num_heads, num_tokens, head_dim)
        queries = queries.transpose(1, 2) # (b, num_heads, num_tokens, head_dim)
        values = values.transpose(1, 2) # (b, num_heads, num_tokens, head_dim)
        
        attn_scores = queries @ keys.transpose(2, 3) # 各ヘッドドット積を計算
        
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens] # マスクをトークン数で切り捨て
        attn_scores.masked_fill_(mask_bool, -torch.inf) # Attention スコアを埋めるためにマスクを使う
        
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # ドロップアウトを適用
        
        context_vec = (attn_weights @ values).transpose(1, 2) # (b, num_heads, num_tokens, head_dim)
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out) # self.out に基づいてヘッドを結合
        
        context_vec = self.out_proj(context_vec) # 線形射像を追加 (これなにやってんの？)
        
        return context_vec

行列計算の例

In [44]:
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],
                   
                   [[0.0722, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]]) # a の形状は (1, 2, 3, 4) で 1つのバッチ、2つのヘッド、3つのトークン、4つの埋め込み次元を持つ
print(a.transpose(2, 3)) # 転置は次元 2 と 3 を入れ替え。トークンと埋め込み次元の部分のみの転置をうまく行っている
print()
print(a @ a.transpose(2, 3))

tensor([[[[0.2745, 0.8993, 0.7179],
          [0.6584, 0.0390, 0.7058],
          [0.2775, 0.9268, 0.9156],
          [0.8573, 0.7388, 0.4340]],

         [[0.0722, 0.4066, 0.4606],
          [0.3565, 0.2318, 0.5159],
          [0.1479, 0.4545, 0.4220],
          [0.5331, 0.9737, 0.5786]]]])

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4384, 0.6983, 0.5880],
          [0.6983, 1.3737, 1.0620],
          [0.5880, 1.0620, 0.9912]]]])


MutiHeadAttention クラスの使い方

In [45]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


In [46]:
mha = MultiHeadAttention(768, 768, 1024, 0.0, num_heads=12)